In [ ]:
!pip install catboost

In [36]:
import pandas as pd
from yargy import Parser, rule, and_, or_
from yargy.predicates import gram
from yargy.interpretation import fact, attribute
from yargy.predicates import normalized, dictionary
from yargy.pipelines import morph_pipeline
from yargy.relations import main
from yargy.predicates import in_
from tqdm import tqdm
import numpy as np
import warnings
import pymorphy2
from gensim.models import Word2Vec as w2v

from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score
from nltk.corpus import stopwords
import pickle

warnings.filterwarnings('ignore')

stopwords_ru = stopwords.words("russian")
morph = pymorphy2.MorphAnalyzer()

In [37]:
Keywords = [
    'окружность', 'высота', 'треугольникabc', 'хорда', 'четырехугольник',
    'точка', 'касательная', 'угол', 'пересечение', 'прямая', 'середина',
    'центр', 'треугольник', 'пересекаться', 'лежать', 'проходить',
    'различный', 'закрасить', 'делиться', 'сумма', 'существует',
    'больший', 'известный', 'клетка', 'каждый', 'цвет', 'доска',
    'малый', 'таблица', 'выбрать', 'найдется', 'фишка',
    'добиться', 'стоить', 'получить',
    'операция', 'написать', 'соседний',
    'количество', 'оказаться', 'стакан','ход',
    'разрешаться', 'многочлен', 'функция', 'уравнение', 'коэффициент',
    'кубический', 'неравенство', 'равенство', 'член',
    'трехчлен', 'равный', 'известный', 'график', 'целый',
    'положительный', 'действительный', 'сумма', 'натуральный',
    'делиться', 'существовать', 'произведение'
    'неравенство', 'группа', 'квадратный', 'команда',
    'положительный', 'неизвестная', 'разделить', 'многочлен',
    'коэффициент', 'расставить', 'число', 'клетка', 'функция',
    'цвет', 'составить', 'член', 'буква', 'равный', 'произведение',
    'сумма', 'соседний', 'состоять', 'известный', 'ровно', 'получить',
    'количество', 'равенство', 'целый', 'различный', 'порядок',
    'делиться', 'график', 'натуральный', 'уравнение', 'существовать',
    'человек', 'действительный', 'город', 'соединить', 'трехчлен',
    'корень', 'множители', 'способ', 'сколько',
    "сумма", "каждый", "коробка",
    "малый", "белый","больший", "набор",
    "бить", "отметить", "больший количество",
    "больший", "шахматный",
    "клетка", 'квадрат', 'остальные', 'доска',
    'число', 'сумма', 'квадрат', 'оказаться',
    'черный', 'отметить', 'остаться',
    'набор', 'записать', 'возможный', 'какой',
    'какой малый', 'какой больший', 'больший количество',
    'шахматный доска', 'возможный', 'малый возможный',
    'найти малый', 'какой минимальный', 'минимальный',
    'максимальный', 'какой максимальный', 'последовательность',
    'получить', 'король', 'размер', 'клетка'
    'некоторый', 'решить', 'сложение', 'порядок',
    'нечетный', 'цифра', 'оканчивается', 'четный',
    'целый', 'число делиться', 'простой', 'натуральный число',
    'делится', 'целый число', 'mod', 'простой число',
    'движение', 'город соединить', 'проехать',
    'связанный', 'каждый город', 'вершина',
    'добраться', 'дорога', 'страна', 'команда',
    'человек', 'граф', 'соединить', 'город', 'знакомый', 'провод'
    
] 
Case = fact('Case', ['name'])

In [38]:
def make_topic_one_to_one(topic: list):
    """
    Функция для формирования pipline слова
    """
    return morph_pipeline(topic).interpretation(
          Case.name.const(topic[0])
      ).interpretation(
          Case
      )

def add_keywords(df, keywords, column_name):
    """
    Функция, которая добавляет ключевые слова в dataset
    """
    alls_word_pipe = []
    for word in keywords:
        alls_word_pipe.append(make_topic_one_to_one([word]))
        df[word] = np.NaN

    ALL = or_(*alls_word_pipe).interpretation(Case)
    parser = Parser(ALL)

    for ind, elem in enumerate(tqdm(df[column_name])):
        for match in parser.findall(str(elem)):
            df.loc[ind, match.fact] = 1
    df.fillna(0, inplace=True)
    return result_df


def add_keywords(df, keywords, column_name):
    """
    Функция, которая добавляет ключевые слова в dataset
    """
    alls_word_pipe = []
    for word in keywords:
        alls_word_pipe.append(make_topic_one_to_one([word]))
        df[word] = np.NaN

    ALL = or_(*alls_word_pipe).interpretation(Case)
    parser = Parser(ALL)

    for ind, elem in enumerate(tqdm(df[column_name])):
        for match in parser.findall(str(elem)):
            df.loc[ind, match.fact] = 1
    df.fillna(0, inplace=True)
    return df


with open('name-to-name.pickle', 'rb') as file:
    name_to_name_dict = pickle.load(file)


# Чистим ненужные символы
def replace_symbol(text):
    text = re.sub(r'[\d′‖√∠ω№◦⊥→×]', lambda match: name_to_name_dict[match.group()], text)
    return text


def del_garbage(text):
    return text.strip('.,)?.<*&^%$#@!~')\
               .replace(u'\u00A0', ' ')\
               .replace(u'\x0c', ' ')\
               .replace('\n', '').replace('\t', '')\
               .replace('\r', '')


# Убираем пунтктуацию
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)


# Лемматизируем текст
def lemmatize(doc, stopwords=stopwords_ru):
    tokens = []
    for token in doc.split():
        if token and token not in stopwords:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            tokens.append(token)
    return ' '.join(tokens)


# Финальная функция обработки текста
def preprocessing_text(text):
    text = del_garbage(text).lower()
    text = lemmatize(text)
    text = replace_symbol(text)
    return text


def vect_tfidf(vectorizer, text):
    """
    Функция, которая добавляет векторное представлние текста с помощью модели vectorizer
    """
    return vectorizer.transform([text]).toarray()


def add_tf_idf(data, X_columns='new_task'):
    """
    Функция, которая векторизует датасет c помощью tf-idf
    """
    vectorizer = read_models("vectorizer.pickle")
    train_title_tfidf = np.vstack(data[X_columns].apply(lambda x: vect_tfidf(vectorizer, x)))
    for i in range(len(vectorizer.get_feature_names_out())):
        data[f"Tiidf title f.{i + 1}"] = train_title_tfidf[:, i]

    return data


def add_embedings(data, X_columns='new_task'):
    """
    Функция, которая векторизует датасет c помощью word2vec
    """
    model = w2v.load('word2vec_math_text.model')
    for i in range(100):
      data[f'vector_{i}'] = 0

    for j, text in enumerate(train[X_columns]):
      vec = np.zeros(100)
      lens = 0
      for word in word_tokenize(text):
          try:
            vec += model.wv[word]
            lens += 1
          except KeyError:
            continue
    vec /= lens
    for i in range(100):
        data.iloc[j, data.shape[1]-100+i] = vec[i]
    return data

def add_tokens(data, X_columns='new_task'):
    """
    Функция, которая векторизирует dataset с помощью tensorflow
    """
    vocab_size = 20000
    trunc_type = 'post'
    padding_type = 'post'
    embedding_dim = 128
    max_length = 120
    oov_tok = '<OOV>'
    
    tokenizer = read_models("tokenizer.pickle")
    
    train_sequences = tokenizer.texts_to_sequences(train[X_columns])
    train_padded = pad_sequences(train_sequences, maxlen=max_length,
                                 padding=padding_type, truncating=trunc_type)
    for i in range(max_length):
        data[f"Tokens f.{i + 1}"] = train_padded[:, i]
    train.fillna(0, inplace=True)
    try:
      data.drop(columns=[None], inplace=True)
    except KeyError as err:
        pass


def read_models(path):
    with open(path, 'rb') as file:
        return pickle.load(file)

In [39]:
data = pd.read_csv('preprocessing_data_big.csv', sep='@').drop(columns=['Unnamed: 0', 'index'])

# Обучение

In [40]:
data_with_keywords = add_keywords(df=data, keywords=Keywords, column_name='task')

100%|██████████| 5803/5803 [26:46<00:00,  3.61it/s]  


In [41]:
data_with_keywords.to_csv('data_with_keywords.csv', sep='@')

In [ ]:
train = data_with_keywords.copy(deep=True)
data = add_tf_idf(train, X_columns='new_task')
data = add_tokens(train, X_columns='new_task')
data = add_embedings(train, X_columns='new_task')

In [ ]:
data.drop(columns=['task', 'new_task'])

In [ ]:
#### class StackingModel(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.final_models = XGBClassifier()
        self.models_fit = [(Ridger, X_test.iloc[:, 100:100+4146]),
                           (LogisticRegression, X_test.iloc[:, :4146]),
                           (SGDr, X_test.iloc[:, :4146]),
                           (SVC, X_test.iloc[:, 100:100+4146]),
                           (KNN, X_test.iloc[:, 100:100+4146]),
                           (RandomForest, X_test.iloc[:, 100:100+4146]),
                           (XGB, X_test.iloc[:, 100:100+4146]),
                           (CatBoost, X_test.iloc[:, 100:100+4146])]
        

    def fit(self, X, y):
        X_train, tr_2, tr_3, X_val, y_train, y_val = train_val_split(X, y)
        val_2 = X_val.iloc[:, 114].to_frame()
        val_3 = X_val.iloc[:, 115:]
        for ind, model in enumerate(self.models):
            if ind == 0 or ind == 3:
              model.fit(tr_3, y_train)
            elif ind == 4:
              model.fit(tr_2, y_train)
            else:
              model.fit(X_train, y_train)

        print('Models trained')
        stack_df = self.transform(X_val)
        self.regressor_final.fit(stack_df, y_val)
        return self

    def transform(self, X):
        stack_df = pd.DataFrame({'lg': self.models[0].predict(X.iloc[:, 115:]),
                                'rig': self.models[1].predict(X),
                                'lasso': self.models[2].predict(X),
                                'bayes': self.models[3].predict(X.iloc[:, 115:]),
                                'svr': self.models[4].predict(X.iloc[:, 114].to_frame()),
                                'knn': self.models[5].predict(X),
                                'free': self.models[6].predict(X),
                                'fr': self.models[7].predict(X),
                                'grboost': self.models[8].predict(X),
                                'xgboost':self.models[9].predict(X),
                                'catboost': self.models[10].predict(X),
                                'MolLogP': list(X.MolLogP)})
        return stack_df


    def predict(self, X):
        stack_df = self.transform(X)
        return self.regressor_final.predict(stack_df)


    def train_val_split(X, y):
        X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.7, random_state=42)
    
        tr_2 = X_train.iloc[:, 114].to_frame()
        tr_3 = X_train.iloc[:, 115:]
    
        return X_train, tr_2, tr_3, X_val, y_train, y_val


    def number_of_atoms(atom_list, df):
        for i in atom_list:
            df['num_of_{}_atoms'.format(i)] = df['mol'].apply(lambda x: len(x.GetSubstructMatches(Chem.MolFromSmiles(i))))
    
    
    def ecfc_molstring(molecule, radius=3, size=1024):
        arr = np.zeros((1,), dtype=int)
        DataStructs.ConvertToNumpyArray(
            AllChem.GetHashedMorganFingerprint(molecule, radius, size, useFeatures=False),
            arr,
        )
        return arr
    
    def mols2features(mols):
        return np.array([ecfc_molstring(mol) for mol in mols])
    
    def CalcDescriptors(data):
        df = pd.DataFrame({name: [] for name in descList})
        result = np.asarray([calculator.CalcDescriptors(mol) for mol in data.mol]).transpose()
        for i, descriptor in enumerate(descList):
            df[descriptor] = result[i]
        return df

# Предсказание

In [ ]:
model_ml = read_models("model_ml")
def get_ml(text):
    data = pd.DataFrame({'text': text,
                         'proc_text': preprocessing_text(text))
    
    data = add_keywords(df=data, keywords=Keywords, column_name='text')
    data = add_tf_idf(data, X_columns='text')
    data = add_tokens(data, X_columns='text')
    data = add_embedings(data, X_columns='text')
    return model_ml.predict_proba(data)